In [ ]:
import numpy as np
import pandas as pd
import cv2

content_path = '../input/neuralstyletransferdata/Content_Kaushik.jpg'
style_path = '../input/neural-style-transfer-data/Style.jpeg'

In [ ]:
import tensorflow as tf
# import tf.keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras.applications import vgg19
from tensorflow.keras.models import Model

tf.config.run_functions_eagerly(True)

In [ ]:
img_nrows, img_ncols = 784, 784
def preprocess_image(path):
    img = load_img(path, target_size = (img_nrows, img_ncols))
    img = img_to_array(img) #Converts a PIL Image instance to a Numpy array.
    img = np.expand_dims(img, axis = 0)
    img = vgg19.preprocess_input(img)
    return img

In [ ]:
def gram_matrix(ip_tensor):
    t = tf.transpose(ip_tensor, (2,0,1))    # making the input tensor as n_c x n_h x n_w
    features = tf.reshape(t, (tf.shape(t)[0], -1)) # making the tensor to n_c x (n_h * n_w) dimensional
    gram_matrix = tf.matmul(features, tf.transpose(features))
    return gram_matrix

def compute_style_loss (style_img, generated_img):
    S = gram_matrix(style_img)
    G = gram_matrix(generated_img)
    ch = 3
    size = (img_nrows * img_ncols)
    J_Style = K.sum(K.square(S - G))/ (4.0 * (ch ** 2) * (size ** 2))
    return J_Style

def compute_content_loss(content_img, generated_img):
    J_Content = K.sum(K.square(generated_img - content_img ))
    return J_Content

In [ ]:
def compute_loss(generated_img, content_img, style_img):
    input_tensor = K.concatenate([content_image, style_image, generated_img], axis = 0)
    feature = feature_extractors(input_tensor)
    loss = K.variable(0.0)
    layer_feature = feature[content_layer]

    content_img_feature = layer_feature[0,:,:,:]
    generated_img_feature = layer_feature[2,:,:,:]

    loss = loss + content_weight * compute_content_loss(content_img_feature, generated_img_feature)

    for layer_name in style_layers:
        layer_feature = feature[layer_name]
        style_img_feature = layer_feature[1,:,:,:]
        generated_img_feature = layer_feature[2,:,:,:]

        loss = loss + (style_weight/len(style_layers)) * compute_style_loss(style_img_feature, generated_img_feature)

    return loss


"""Error in the above function"""

In [ ]:
@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

In [ ]:
def deprocess_image(x):

    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
# tensor representations of the input image.
content_image = K.variable(preprocess_image(content_path))
style_image = K.variable(preprocess_image(style_path))

combination_image = K.variable(preprocess_image(content_path))

""" Concatenate the Content Image, Style Image and Resultant Image: Results a shape [3, 512, 512, 3]"""
# input_tensor = K.concatenate([content_image, style_image, combination_image], axis = 0)

# model = vgg19.VGG19(weights = 'imagenet',  include_top = False)
model = vgg19.VGG19(weights= '../input/vgg-weights/vgg19_notop.h5', include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractors = tf.keras.Model(inputs = model.inputs, outputs = outputs_dict)

content_layer = 'block5_conv2'
style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']



style_weight = 1e-6
content_weight = 2.5e-8

optimizer = tf.keras.optimizers.SGD(
  tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=300.0, decay_steps=500, decay_rate=0.96
  )
)

result_prefix = "Lab_generated"

iterations = 20000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, content_image, style_image
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print("Iteration %d: loss=%.2f" % (i, loss))
        img = deprocess_image(combination_image.numpy())
        fname = result_prefix + "_at_iteration_%d.png" % i
        tf.keras.preprocessing.image.save_img(fname, img)